In [94]:
import streamlit as st
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI, OpenAI
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate

In [66]:
loader = DirectoryLoader('docs')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
docs = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(openai_api_key=st.secrets['OPENAI_API_KEY'])
vectordb = FAISS.from_documents(docs, embeddings)
retriever = vectordb.as_retriever()

In [100]:
rel_docs = retriever.get_relevant_documents("What University has Radek finished?")

In [111]:
format_chat_history = lambda messages: '\n'.join([f'{m["role"]}: {m["content"]}' for m in messages])
format_additional_information = lambda documents: "\n\n".join([str(item.metadata) + "\n" + item.page_content for item in documents])

In [110]:
template = """You are a chatbot that helps people to get information about Radek Szostak. Radek is a data scientist and a machine learning engineer. He is also a co-founder of the LangChain project. He is a very friendly and helpful person. He is always ready to help people. He is also a very good teacher. He is always ready to teach people. He is also a very good mentor. He is always ready to mentor. You have access to chat history in order to keep context of the conversation. You are also provided with additional information where you can search for answers.

% start additional information %
{additional_information}
% end additional information %

% chat history %
{chat_history}
assisstant:
"""
prompt = PromptTemplate(input_variables=['chat_history'], template=template)
client = OpenAI(model="gpt-3.5-turbo-instruct", api_key=st.secrets["OPENAI_API_KEY"], streaming=True)
messages = []
while True:
    user_message = input("You: ")
    messages.append({"role": "user", "content": user_message})
    chat_history = format_chat_history(messages)
    additional_information = format_additional_information(retriever.get_relevant_documents(chat_history))
    
    response = ""
    for chunk in client.stream(prompt.format(chat_history=chat_history, additional_information=additional_information)):
        response += chunk
        print(chunk, end="", flush=True)
    messages.append({"role": "assistant", "content": response})

NameError: name 'format_chat_history' is not defined

In [58]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm = ChatOpenAI(),
    retriever = vectordb.as_retriever(),
    verbose = True,
)

In [59]:
qa_chain.invoke({'question': 'Who are you?', 'chat_history': []})#, 'chat_history': []})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
university teacher and member of an international research group.

Interests

Python

environments. Nearly three years of experience as a software developer in a commercial company.
Human: Who are you?

> Finished chain.

> Finished chain.


{'question': 'Who are you?',
 'chat_history': [],
 'answer': 'I am an AI assistant designed to provide information and answer questions. I am not a specific person, but I can assist with a wide range of topics. Is there something specific you would like to know or discuss?'}

In [56]:
qa_chain.invoke({'question': 'What was the previous question?'})#, 'chat_history': []})

ValueError: Missing some input keys: {'chat_history'}

# ---

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain
from langchain.llms import OpenAI

template = """
You're a chatbot representing a talented individual looking to share insights with recruiters. Start by introducing the candidate's background, skills, and interests. Provide an overview of their professional journey, highlighting key experiences, projects, and achievements. Emphasize the candidate's strengths, such as their problem-solving abilities, creativity, and dedication to continuous learning. Incorporate examples of successful collaborations, leadership roles, and contributions to previous teams or projects. Encourage the recruiter to ask questions for further clarification or to explore specific areas of expertise. Aim to create a conversational tone that is engaging and informative, showcasing the candidate's personality and enthusiasm for their field.

### Candidate Description
{cantidate_description}

### Chat History
{chat_history}

###
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["cantidate_description", "chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(
    llm=OpenAI(openai_api_key=st.secrets['OPENAI_API_KEY']), 
    prompt=prompt, 
    verbose=True, 
    memory=memory
)

In [ ]:
llm_chain.predict(context = retriever, human_input="Is an pear a fruit or vegetable?")